함수형 프로그래밍 패러다임에 있어 매우 중요한 특징 하나는 고차 함수다. 고차 함수는 다른 함수를 인자로 받거나 함수를 결과로 반환하는 함수다. 파이썬은 여러 종류의 고차 함수를 지원한다는 것에 대해 살펴보고, 그러한 함수를 논리적으로 확장한 것을 다룰 것이다. 앞으로 살펴보겠지만, 고차 함수에는 다음과 같이 세 가지 유혀잉 존재한다.

* 인자 중 하나로 함수를 요구하는 함수
* 함수를 반환하는 함수
* 함수를 인자로 받고, 함수를 반환하는 함수

함수를 만들어 내는 함수라는 개념은 조금 이상하게 들릴수도 있다. 하지만 Callable 클래스의 객체에 대해 공부한다면 호출 가능한 객체를 반환하는 함수를 보게 된다. 그러한 함수는 다른 함수를 만들어 내는 함수의 한 가지 예다.

함수를 인자로 받아 다른 함수를 만들어 내는 함수의 예로는 복잡한 Callable 클래스나 함수 데코레이터가 있다.

앞에서 설명한 여러 컬렉션 함수의 고차 함수 버전이 있었으면 할 때가 있다. 이번 장에서는 커다란 튜플에서 특정 필드를 추출하여 축약을 수행하는 출약(추출()) 설계 패턴에 대해 살펴본다. 또한 이러한 공통 컬렉션 처리 함수를 직접 정의하는 방법에 대해 살펴본다.

이번 장에서는 다음과 같은 함수를 살펴본다.

* max()와 min()
* 고차 함수를 호출할 때 편하게 쓸 수 있는 람다식
* map()
* filter()
* iter()
* sorted()

max()와 min() 함수는 축약이다. 컬렉션으로부터 값을 하나 만들어 낸다. 다른 함수들은 매핑이다. 이들은 하나의 값으로 축약시키지 않고 컬렉션을 변환한다.

### max()와 min()을 사용해 최댓값, 최솟값 알아내기

max()와 min() 함수는 두 가지 동작이 가능하다. 이들은 컬렉션에 적용할 수 있는 단순한 함수지만, 또한 고차 함수이기도 하다. 이들의 기본적인 동작은 다음과 같다.

In [1]:
max(1,2,3)

3

In [2]:
max((1,2,3))

3

두 함수 모두 임의의 길이 인자를 받는다. 이들은 유일한 인자로 시퀀스뿐만 아니라 반복 가능 객체도 받도록 설계됐고, 인자로 받은 반복 가능 객체의 원소 중 최댓값을 찾는다.

좀 더 복잡한 동작도 가능하다. 4 장에서 본 여행 경로 데이터를 생각해보자. 거기서 다음과 같은 튜플의 시퀀스를 만들어 내는 함수를 살펴봤다. 

In [7]:
trip = (((37.54, -76.33), (37.84, -76.27), 17.72), ((37.84, -76.27), (38.33, -76.45), 30.73), ((38.33, -76.45), (38.84, -76.53), 31.07))

각 3-튜플에는 시작 지점의 위치, 끝 지점의 위치 거리 값이 들어 있다. 위치는 위도와 경도의 쌍이다. 이 시퀀스에서 최대와 최소 거리를 찾는 데에는 세 가지 방법이 있다. 각각은 다음과 같다.

* 거리는 제너레이터 함수로 뽑아낸다. 그 함수는 오직 거리만을 돌려주고, 해당 구간의 시작과 끝 위치는 버린다. 최댓값이나 최솟값 구간의 위치 정보를 가지고 추가 처리를 해야 하는 경우에는 이러한 방식이 잘 들어맞지 않을 것이다. 

* 풀기(처리(감싸기())) 패턴을 사용한다. 이렇게 하면, 최대, 최소 거리가 있는 구간을 얻을 수 있다. 그로부터 거리만 필요하다면 거리만을 뽑아낼 수도 있다. 구간을 나타내는 두 위치 정보를 사용해 어떤 구간이 최대 또는 최소인지를 파악할 수 있따.

* max()와 min()을 고차 함수로 사용한다.


다음은 풀기(처리(감싸기())) 패턴을 사용한 것이다. 실제로토 wrap()와 unwrap() 이라는 이름의 함수를 정의하여 패턴이 어떻게 동작하는지 명확히 이해할 수 있게 했다.

In [8]:
def wrap(leg_iter):
    return ((leg[2], leg) for leg in leg_iter)

def unwrap(dist_leg):
    distance, leg = dist_leg
    return leg

long, short = unwrap(max(wrap(trip))), unwrap(min(wrap(trip)))
print(long, short)

((38.33, -76.45), (38.84, -76.53), 31.07) ((37.54, -76.33), (37.84, -76.27), 17.72)


마지막으로 가장 중요한 형태는 max()와 min() 고차 함수 기능을 사용한다. 이를 위해 먼저 도우미 함수를 정의하고 그 함수를 사용하여 구간의 컬렉션을 원하는 요약 형태로 축약한다. 다음을 살펴보자.

In [9]:
def by_dist(leg):
    lat, lon, dist = leg
    return dist
long, short = max(trip, key=by_dist), min(trip, key=by_dist)
print(long, short)

((38.33, -76.45), (38.84, -76.53), 31.07) ((37.54, -76.33), (37.84, -76.27), 17.72)


by_dist() 함수는 구간 leg 튜플의 세 원소를 분리하여 거리를 반환한다. 이를 max(0와 min() 함수에 사용한다.

max()와 min() 함수는 모두 반복 가능 챛ㅇ게와 함수를 인자로 받는다. 파이썬의 모든 고차 함수에서는 키워드 매개변수 key=를 사용해 필요한 키 값을 뽑아내도록 할 수 있다.

max() 함수가 key로 지정한 함수를 어떻게 사용하는지 다음 코드가 개념화해 보여준다.

In [14]:
wrap = ((key(leg), leg) for leg in trip)

### 파이썬의 람다 식 사용하기

많은 경우 도우미 함수를 따로 정의하는 것은 필요 이상의 코드를 작성하게 된다. key 함수의 핵임을 식 하나로 표현할 수 있는 경우가 자주 있다. 그러한 경우조차 식 하나를 일일히 def문과 return문으로 감싸는 것은 낭비같아 보인다.

파이썬은 람다 형식을 사용해 고차 함수를 쉽게 사용할 수 있게 해준다. 
다음은 간단한 lambda 식을 사용해 key를 지정하는 것을 보여준다.

In [17]:
long, short = max(trip, key=lambda leg: leg[2]), min(trip, key=lambda leg: leg[2])
print(long, short)

((38.33, -76.45), (38.84, -76.53), 31.07) ((37.54, -76.33), (37.84, -76.27), 17.72)


사용한 lambda에는 시퀀스의 원소가 주어질 것이다. 여기서는 trip의 각 구간을 표현하는 3-튜플이 람다에 전달될 것이다. lambda 인자 변수 leg에는 3-튜플이 들어가고, 식 leg[2]를 평가하여 거리 정보만을 선택한다.

람다를 재활용할 필요가 없는 경우에는 이러한 형태가 가장 이상적이다. 하지만 람다 객체를 재활용해야 하는 경우도 자주 있다. 복사해 붙여넣기가 좋은 생각이 아니라면, 대안에는 무엇이 있을까?

람다 함수를 정의할 수 있다.
또는 다음과 같이 람다를 변수에 대립할 수도 있다.

In [18]:
start = lambda x: x[0]
end = lambda x: x[1]
dist = lambda x:x[2]

람다는 호출 가능 객체이기 때문에 함수처럼 사용할 수 있다.

파이썬에서 튜플의 우너소에 대해 의미 있는 이름을 정의하는 방법은 namedtuple을 사용하는 것과 람다를 사용하는 두 가지가 있다. 두 방법은 동등하다. 

이 예제를 확장하기 위해 시작점과 끝점에서 위도와 경도를 얻는 방법을 살펴본다. 람다를 몇 개 더 정의하면 그렇게 할 수 있다.

다음은 앞에서 본 인터프리터 세션을 계속 진행한 것이다.

In [20]:
leg = ((27.15, -80.19), (29.19, -81), 129)

In [21]:
start(leg)

(27.15, -80.19)

In [22]:
lat = lambda x: x[0]
lon = lambda x: x[1]

In [23]:
lat(start(leg))

27.15

람다를 사용하는 것이 이름 있는 튜플을 사용하는 것보다 더 쓸모 있는지는 분명하지 않다. 필드를 뽑아내기 위해 여러 람다를 정의하는 것은 이름 있는 튜플을 정의하는 것보다 더 많은 코드가 필요하다. 반면, 람다를 사용하면 전위 형식의 함수 호출 표기법을 사용할 수 있기 때문에 함수형 프로그래밍을 하는 상황에서는 그 편이 더 나을 것이다. 더 중요한 것은, 나중에 sorted() 예제에서 볼 수 있는 것 처럼, sorted(), min(), max() 등의 함수에서 이름 있는 튜플보다 람다를 더 효과적으로 사용할 수 있다는 점이다.

### 람다와 람다 대수
순수 함수 언어에 대한 책에서는 람다 대수와 하스켈 커리가 발명한 커링이라는 기법에 대해 설명할 필요가 있을 것이다. 하지만 파이썬은 그러한 람다 대수와는 거리가 멀다. 파이썬의 함수는 단일 인자의 람다 형식으로 환원되지 않는다.

커링을 functools.partial 함수를 사용해 구현할 수 있다.

### map() 함수를 사용해 함수를 컬렉션에 적용하기

스칼라 함수는 값을 정의역으로부터 치역으로 변환한다. math.sqrt() 함수를 살펴보면, float 값 x를 다른 float 값인 $y^2 = x$인 $y=sqrt(x)$로 매핑한다. 정의역은 양수로 제한된다. 이 매핑은 계산으로 이뤄질 수도 있고, 표를 사용한 보간법으로 이뤄질 수도 있다.

map() 함수는 비슷한 개념을 표현한다. 이 함수는 한 컬렉션을 다른 컬렉션으로 매핑한다. 이 함수는 주어진 함수를 원래 컬렉션의 모든 원소에 적용하여 결과 컬렉션을 만들어 내는 것을 보장한다. 이는 내장 함수를 테이터의 컬렉션에 대해 적용하는 이상적인 방법이다.

처음으로 살펴볼 예제는 텍스트 블록을 가지고 수의 시퀀스를 얻는 것이다. 다음 텍스트를 살펴보자.

In [24]:
text = """2 3 5 7 11 13 17 19 23 29 31 37 41 43"""

이 텍스트를 다음 제너레이터 식을 사용해 재구성할 수 있다.

In [25]:
data = list(v for line in text.splitlines() for v in line.split())

In [26]:
data

['2',
 '3',
 '5',
 '7',
 '11',
 '13',
 '17',
 '19',
 '23',
 '29',
 '31',
 '37',
 '41',
 '43']

여기서 int() 함수를 각 문자열에 적용해야 한다. 이러한 경우 map() 함수가 멋지게 역할을 해낼 수 있다.

In [27]:
list(map(int, data))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43]

map() 함수는 int() 함수를 data 컬렉션의 모든 값에 대해 호출한다. 결과는 문자열의 시퀀스가 아니라 수의 시퀀스다.

map() 함수는 반복 가능 객체를 돌려준다. map() 함수는 모든 반복 가능 객체를 처리할 수 있다.

여기서 핵심 아이디어는 map() 함수를 사용하면 파이썬의 함수를 컬렉션의 모든 원소에 적용할 수 있다는 것이다. 이러한 매핑 처리에 사용할 수 있는 내장 함수가 많이 있다.

### map()에 람다 형식 사용하기
여행 경로의 각 구간 거리를 해리에서 일반 마일로 바꾸고 싶다고 가정하자. 각 경로의 거리에 1.150780를 곱하면 된다.
map()  함수를 사용해 다음과 같이 이러한 계산을 수행할 수 있다.

In [31]:
map(lambda x: (start(x), end(x), dist(x) *6076.12/5280), trip)

In [32]:
list(_)

[((37.54, -76.33), (37.84, -76.27), 20.391826969696968),
 ((37.84, -76.27), (38.33, -76.45), 35.36347871212121),
 ((38.33, -76.45), (38.84, -76.53), 35.754744015151516)]

map 함수와 제너레이터 식 사이의 중요한 차이점은 map() 함수가 제너레이터식보다 더 빠른 경향이 있다는 점이다. 속도 향상은 보통 20퍼센트 정도다.

### map()을 여러 시퀀스에 활용하기

때로 두 컬렉션에 있는 데이터를 각각 짝지워 사용해야 할 경우가 있다.
map(function, zip(one_iterable, another_iterable))

두 반복 가능 개체로부터 인자 튜플을 만들어 내서 함수를 그 튜플에 적용했다. 이를 다음과 같이 적용해 볼 수도 있다.

(function(x, y) for x, y in zip(one_itrable, another_iterable))

이러한 아이디어 전체를 일반화시킨다면 다음과 같을 것이다.

In [34]:
def star_map(function, *iterables):
    return (function(*args) for args in zip(*iterbale))

하지만 이보다 더 나은 접근 방법이 이미 존재한다. XML 파일에서 여행 지점의 목록을 추출한 것을 봤다. 이 지점으로부터, 시작점과 끝점으로 이뤄진 구간의 리스트를 만들 필요가 있었다.

다음은 특별한 반복 가능 객체에 zip()을 적용하여 단순화한 코드다.

In [35]:
waypoints = range(4)
zip(waypoints, waypoints[1:])

In [36]:
list(_)

[(0, 1), (1, 2), (2, 3)]

평면적인 리스트에서 튜플의 시퀀스를 끌어냈다. 각 튜플은 인접한 두 값을 포함할 것이다. zip() 함수는 더 짧은 목록이 끝나면 제대로 수행을 멈춘다. 하지만 zip(x, x[1:]) 패턴은 실체화한 시퀀스나 range() 함수로 만들어 낸 반복 가능 객체에만 작동한다.

튜플들을 만들고 나면 harversine() 함수를 적용해 튜플의 양 끝 지점 사이의 거리를 계산할 수 있다.

### filter()를 사용해 데이터를 받아들이거나 거부하기

filter() 함수의 목적은 술어라고 불리는 어떤 의사결정 함수를 주어진 컬렉션의 모든 원소에 적용하는 것이다. True라는 결과가 나오면 그 값을 통과시키고, 그렇지 않으면 그 값을 거부한다. itertools 모듈에는 이와 비슷한 목적의 filterflase()라는 함수가 있다. 

이 함수를 사용하면 여행 데이터에서 50해리 이하의 구간만으로 이뤄진 하위 집합을 구할 수 있을 것이다.

In [37]:
long = list(filter(lambda leg: dist(leg) >= 50, trip))

여기서 사용한 술어 람다는 거리가 긴 구간에 대해 True를 반환하며, 그러한 구간은 결과 리스트에 들어간다. 거리가 짧은 구간은 거부될 것이다. 이 검사를 통화가는 구간은 14개다.

In [40]:
filter(lambda x: x%3 ==0 or x%5==0, range(10))

간단한 람다를 정의하여 어떤 수가 3이나 5의 배수인지 검사했다. 그 함수를 반복 가능 객체인 range(10)에 적용했다. 결과는 규칙을 만족하는 수의 반복 가능한 시퀀스다.

이 람다가 True를 반환하는 수는 [0, 3, 5, 6, 9]이다. 따라서 그 안에 들어 있는 수는 결과 시퀀스에 남는다. 하지만 람다가 False를 반환하는 수들은 제외될 것이다.

다음처럼 이를 제너레이터 식 안에서 수행할 수도 있다.

In [41]:
list(x for x in range(10) if x%3==0 or x%5==0)

[0, 3, 5, 6, 9]

이는 range(10)에 속하면서 x%3==0 또는 5인 x로 이뤄진 컬렉션을 만들고 있다. filter 함수와 수학적인 집합 조건 제시법 사이에는 매우 우아한 유사성이 존재한다. 

### filter()를 사용해 이상 값 식별하기
앞 장에서 우리는 평균이나 표준편차를 계산하거나 값을 정규화하는 여러 유용한 통계 함수를 정의했다. 이러한 함수를 사용하여 여행 정보에서 이상 값을 찾아낼 수 있다. 여행의 각 구간에 mean()과 stdev() 함수를 적용하여 모집단의 평균과 표준편차를 얻는 것이다.

그후 z() 함수를 호출하여 각 구간을 정규화한 값을 구할 수 있다. 정규화한 값이 3보다 크다면, 해당 데이터는 평균에서 심하게 벗어난 것이다. 이러한 이상 값을 버린다면, 항구에서 잘못 보고했거나 측정 시 오류가 있었던 데이터가 들어갈 가능성을 낮출 수 있을 것이다.

다음은 이 문제를 처리하는 방법을 보여준다.

In [53]:
from stats import mean, stdev, z
dist_data = list(map(dist, trip))
μ_d = mean(dist_data)
σ_d = stdev(dist_data)
outlier = lambda leg: z(dist(leg), μ_d, σ_d) > 3
print("Outliers", list(filter(outlier, trip)))

### iter() 함수와 끝을 표시하는 특별한 값 사용하기

내장 iter 함수는 어떤 컬렉션 객체를 기반으로 하는 반복자를 만들어 낸다. 이를 사용해 어떤 컬렉션을 감싸 반복자를 만들 수 있다. 대부분의 경우, for문을 사용하여 이를 암묵적으로 처리할 수 있다. 몇몇 경우에는 반복자를 명시적으로 만들어 컬렉션의 첫 원소와 나머지 부분을 분리하고 싶을 수도 있다. 이러한 함수는 또한 어떤 호출 가능 객체나 함수가 만들어 낸 값에서 끝을 표시하는 특별한 값이 반환되기 전까지 한 줄씩 파일을 읽는 read() 메서드일 수 있다. 

iter()에 호출 가능한 함수를 제공하려면, 그 함수 내에서 상태를 유지해야 하기 때문에 약간 더 복잡해진다. 이러한 감춰진 상태는 열려 있는 파일의 특성 중 하나다. 예를 들어 각각의 read()나 readline() 함수는 파일의 내부 상태를 한 글자 또는 한 줄 더 진행시킨다.

이에 대한 다른 예로는 변경 가능한 컬렉션 객체의 pop() 메서드가 해당 객체의 상태를 바꾸는 것을 들 수 있다. 다음은 pop() 메서드를 사용하는 예다.

In [54]:
tail = iter([1, 2, 3, None, 4, 5, 6].pop, None)
list(tail)

[6, 5, 4]

tail 변수는 리스트 [1, 2, 3, None, 4, 5, 6]에 대한 반복자였다. 이때 리스트에 대한 방문은 pop() 함수를 사용한다. pop()의 기본 동작은 pop(-1)로, 원소를 역순으로 하나씩 빼내는 것이다. 끝을 표현하는 특별한 값이 발견되면 iter()가 반환하는 반복자는 더 이상 값을 내놓지 않는다.

이러한 식의 내부 상태는 가능한 한 피해야 할 것이다. 따라서 이러한 기능을 활용하려고 시도하지는 않을 것이다.

### sorted()를 사용해 데이터 정렬하기
sorted()는 어떠한 반복 가능 객체에도 사용할 수 있다. 다만, 정렬한 결과 list 객체를 만들어 낸다.

### 고차 함수 작성하기

고차 함수에는 다음과 같이 세 가지 종류가 있다는 것을 알 수있다. 

* 인자 중 하나로 함수를 요구하는 함수
* 함수를 반환하는 함수. Callable 클래스는 이러한 경우의 전형적인 예다. 제너레이터 식을 만들어 내는 함수도 고차 함수라고 생각할 수 있다.
* 함수르 인자로 받고, 함수를 반환하는 함수. functools.partial() 함수가 전형적인 예다. 데코레이터도 또 다른 예다.

지금까지 살펴본 고차 함수를 사용하는 간단한 패턴을 확장하여 데이터의 구조를 변환할 것이다. 또한 몇 가지 일반적인 변환에 대해서도 다룰 것이다.

* 객체를 감싸 더 복잡한 객체 만들기
* 복잡한 객체를 구성 요소별로 분해하기
* 구조를 평면으로 펼치기
* 평뮨 시퀀스를 구조화하기

Callable 클래스 객체는 호출 가능한 Callable 객체를 반환하는 함수의 예로 자주 쓰인다. 설정 매개변수를 주입할 수 있는 유연한 함수를 작성하는 방법에서 이를 사용하는 예제를 살펴본다

또한 이번 장에서는 간단한 데코레이터를 소개할 것이다.

### 고차 매핑과 필터 작성하기

파이썬의 내장 고차 함수인 map()과 filter()는 우리가 처리해주길 원하는 대부분의 처리를 일반적으로 감당할 수 있다. 하지만 성능을 더 높이기 위해 이를 일반적으로 최적화하기는 어렵다.

매핑을 기술하는 방법은 세 가지가 있다. 어떤 함수 f(x)와 어떤 컬렉션 객체 C가 있다고 가정해보자. 이를 사용한 매핑을 정의하는 세 가지 거의 동등한 방법이 있는데, 각각은 다음과 같다.

* map() 함수:

    map(f, C)


* 제너레이터 식:

    (f(x) for x in C)


* 제너레이터 함수:

    def mymap(f, C):
        for x in C:
            yield f(x)
    mymap(f, C)

이와 마찬가지로, filter() 함수를 컬렉션 C와 술어 함수 f에 적용하는 데에도 세 가지 방법이 있다.

* map() 함수:

    filter(f, C)


* 제너레이터 식:

    (f(x) for x in C)


* 제너레이터 함수:

    def myfilter(f, C):
        for x in C:
            yield f(x)
    myfilter(f, C)
    
하지만 성능상 다른 점이 있다. map()과 filter() 함수가 가장 빠르다. 더 중요한 것은 각각의 매핑, 필터와 어울리는 확장이 서로 다르다는 것이다. 그 내용은 다음과 같다.

* 각 원소에 좀 더 복잡한 g(x)를 적용하도록 만들거나 컬렉션 C를 처리하기 전에 함수를 적용할 수 있다. 이는 가장 일반적인 방법이며, 앞에서 설명한 세 가지 방식에 모두 적용할 수 있다. 이 부분에 함수적 설계 역량을 집중해야 할 것이다.

* for 루프를 약간 병경할 수 있다. 한 가지 분명히 알 수 있는 변경은 제너레이터 식을 if절로 확장하여 매핑과 걸러내기를 한 연산과 조합하는 것이다. 이와 마찬가지로 mymap()과 myfilter()를 합쳐 매핑과 걸러내기를 동시에 수행하게 만들 수도 있다.

우리가 만들 수 있는 가장 심오한 변화는 루프가 다루는 데이터의 구조를 변경시키는 것이다. 감싸기, 풀기, 펼치기, 구조화하기 등의 설계 패턴은 이미 존재한다

너무 많은 변환을 한 함수로 조합하는 매핑을 설계하는 경우에는 많은 주의를 기울여야 한다. 가능한 한 둘 이상의 개념을 함께 표현하거나간결하지 못한 함수를 만드는 일은 피해야 한다. 파이썬 컴파일러가 최적화해주지 못하기 때문에 함수를 직접 합쳐 느린 애플리케이션을 최적화해야만 한다. 이러한 종류의 최적화는 가능한 피해야 하며, 성능이 떨어지는 프로그램을 프로파일링한 후 최적화 여부를 결정해야 한다.

### 매핑하면서 데이터 풀기
(f(x) for x, y in C)와 같은 구성을 사용할 경우, for문에 여러 가지 대입을 사용해 값이 여럿 들어 있는 튜플을 풀어 함수를 호출하는 것이다. 전체 식은 매핑이다. 이러한 식은 구조를 바꾸고 함수를 적용하기 위한 일반적인 파이썬 최적화 방법 중 하나다.

다음은 매핑을 진행하는 과정에서 데이터를 푸는 것을 보여주는 구체적인 예다.

In [56]:
def convert(conversion, trip):
    return (conversion(distance) for start, end, distance in trip)

이 고차 함수는 다음과 같은 변환 함수에 의해 지원될 수 있을 것이다.

In [58]:
to_miles = lambda nm: nm*5280/6076.12
to_km = lambda nm :nm*1.852
to_nm = lambda nm: nm

이제 이 함수를 사용해 거리를 뽑아내 변환 함수를 적용할 수 있다.

이 convert() 함수는 사용하는 시작-끝-거리 구간 정보와 밀접한 관련이 있다. 왜냐하면 for 루프 안에서 3-튜플을 분해하기 때문이다.

이러한 식으로 매핑하는 과정에서 데이터를 푸는 디자인 패턴을 더 일반적으로 해결할 수 있다. 그 대신 좀 더 복잡해지는 것을 감수해야 한다. 첫째, 다음과 같은 일반적인 분해 함수가 필요하다.

In [59]:
fst = lambda x: x[0]
snd = lambda x: x[1]
sel2 = lambda x: x[2]

복잡한 함수가 단순한 map, filter의 조합보다 성능이 더 나쁠 수도 있다. 일반적으로는 어떤 한 가지 개념을 담고 있거나 소프트웨어를 더 이해하기 쉽게 해주는 경우에만 더 복잡한 함수를 만들어야 한다.

### 매핑하면서 추가 데이터 감싸기

(f(x), x) for x in C)와 같은 요소를 사용한다면 매핑을 적용하면서 여러 값이 들어 있는 튜플을 생성하기 위해 감싸고 있는 것이다. 이러한 방식은 복잡하게 상태가 바뀌는 객체에 의존하지 않고도 여러 번 재계산하는 것을 피하는 이점을 누리면서 새로운 요소에 필요한 결과를 저장하기 위해 사용하는 일반적인 기법이다.

4장의 지점의 경로로부터 구간 데이터를 만들어 내는 코드를 변경하여 감싸는 작업을 별도의 함수로 분리할 수 있다.

In [60]:
def cons_distance(distance, legs_iter):
    return ((start, end, round(distance(start, end), 4)) for start, end in legs_iter)

이 함수는 각 구간을 두 변수 start와 end로 분해한다. 각각을 주어진 distance() 함수에 사용하여 두 지점 사이의 거리를 구한다. 결과는 원래의 두 지점과 계산한 거리를 포함하는 더 복잡한 3-튜플이다.

이제 harversion() 함수로 거리를 계산하도록 원래의 예제를 다음과 같이 재작성 할 수 있다.

In [62]:
path = float_from_pair(lat_lon_kml())
trip2 = tuple(cons_distance(harversine, legs(ter(path))))

제너레이터 식을 고차 함수 cons_distance()로 바꿨다. 이 함수는 함수를 이자로 받을 뿐만 아니라 제너레이터 식을 반환한다.

In [64]:
def cons_distance3(distance, legs_iter):
    return (leg + (round(distance(*leg), 4),) for leg in legs_iter)

### 매핑하면서 데이터 펼치기

### 걸러내면서 데이터 구조화하기

앞의 세 예제에서는 매핑과 처리를 결합했다. 처리와 걸러내는 것을 결합하는 것은 처리와 매핑을 결합하는 것에 비해 그리 표현력을 높여줄 것 같지는 않다.

다음은 반복 가능 객체의 출력을 그룹으로 묶는 함수 중 우리가 더 선호하는 버전을 보여준다.

In [65]:
def group_by_iter(n, iterable):
    row = tuple(next(iterable) for i in range(n))
    while row:
        yield row
        row = tuple(next(iterable) for i in range(n))

이 함수는 iterable에서 가져온 원소를 n개씩 묶은 튜플을 만든다. 튜플에 원소가 있다면, 그 원소들은 결과로 만들어 내는 반복 가능 객체의 일부부능로 내보내진다. 원칙적으로는, 그 다음에 이 함수를 재귀적으로 입력 iterable의 남은 원소에 적용할 수 있다. 파이썬에서는 재귀가 상대적으로 비효율적이기 때문에 이를 while 루프를 명시적으로 활용해 최적화할 수 있다.

### 제너레이터 함수 작성하기

다양한 함수를 제너레이터 식으로 깔끔하게 표현할 수 있다. 실제로, 우리는 거의 대부분의 매핑이가 걸러내기를 제너레이터식으로 수행할 수 있다는 것을 살펴봤다. 같은 작업을 map(), filter(), 다른 제너레이터 함수와 같은 내장 고차 함수를 사용해 처리할 수도 있다. 여러 문장으로 된 제너레이터 함수를 고려하고 있다면, 함수형 프로그래밍을 할 때 도움이 되는 원칙 중 하나인 '상태가 없는 함수 평가;를 놓치고 있는 것은 아닌지 유념해야 한다.

파이썬을 함수형 프로그래밍에 사용한다는 것은 순수한 함수형 프로그래밍과 명령형 프로그래밍이라는 칼날 위를 걷는 것과 같다. 다른 함수적인 대안이 없는 경우 명령형 파이썬 코드를 사용해야 하는 부분을 식별하고 고립시켜야 한다.

### Callable로 고차 함수 만들기

Callable 클래스의 인스턴스를 사용해 고차 함수를 만들 수도 있다. 이는 제너레이터 함수를 만든다는 아이디어로부터 나온 것이다. 우리가 호출 가능 객체를 사용하는 이유는 파이썬의 문장 수준 기능을 활용하기 위해서다. 문장을 사용하는 것과 더불어, 고차 함수를 만들 때 정적인 설정을 적용할 수도 있다.

Callabe 클래스 정의에 있어 중요한 것은 class문으로 만들어지는 클래스 객체에 함수를 만들어 내는 함수가 반드시 정의되어 있어야 한다는 것이다. 보통은 기존 함수를 둘 조합하여 상대적으로 더 복잡한 새로운 함수를 만들 때 callable 객체를 사용한다.

이를 강조하기 위해 다음 클래스를 살펴보자.

In [1]:
from collections.abc import Callable

class NullAware(Callable):
    def __init__(self, some_func):
        self.some_func = some_func
    def __call__(self, arg):
        return None if arg is None else self.some_func(arg)

이 클래스는 NullAware()라는 이름의 함수를 만든다. 그 함수는 새로운 함수를 만들어 내는 고차 함수다. NullAware(math.log)라는 식을 평가하면, 인자 값에 적용할 수 있는 새로운 함수를 만들 수 있다. \__init__() 메서드는 객체에 그렇게 만들어진 함수를 저장한다.

\__call__() 메서드는 결과 함수가 평가되는 방식을 지정한다. 이 예제에서는 예외를 발생시키지 않고 None 값을 매끄럽게 처리할 수 있게 해주는 함수를 만들어 낸다.

이러한 식으로 새로운 함수를 만들고, 나중에 사용하기 위해 그 함수에 이름을 지정해두는 일반적인 방식은 다음과 같다.

In [3]:
import math

null_log_scale = NullAware(math.log)

이는 새로운 함수를 만들어 null_log_scale() 이라는 이름을 부여한다. 그렇게 하고 나면 이 함수를 다른 곳에서 사용할 수 있다. 다음 실행 예를 살펴보라.

In [5]:
some_data = [10, 100, None, 50, 60]
scaled = map(null_log_scale, some_data)
list(scaled)

[2.302585092994046,
 4.605170185988092,
 None,
 3.912023005428146,
 4.0943445622221]

조금 덜 일반적인 접근 방식으로는 다음과 같이 만들어 낸 함수를 식에서 즉시 호출하는 것이 있다.

In [6]:
scaled = map(NullAware(math.log), some_data)
list(scaled)

[2.302585092994046,
 4.605170185988092,
 None,
 3.912023005428146,
 4.0943445622221]

NullAware(math.log)를 평가하면 함수가 만들어진다. 여기서는 그 이름이 없는 함수를 map() 함수에 사용하여 some_data라는 반복 가능 객체를 처리했다.

이 예제의 \__call__() 메서드는 온전히 식의 평가에만 의존한다. 이는 저수준 함수로부터 합성 함수를 정의하는 고상하고 깔끔한 방법이다. 스칼라 함수를 다루는 경우에는 설계 시 고려해야 할 것이 몇 가지 있다. 반복 가능한 컬렉션을 다루는 경우에는 그보다 좀 더 주의를 기울여야 한다.

### 좋은 함수형 설계를 보장하기

파이썬의 객체를 사용하는 경우, 상태가 없는 함수형 프로그래밍이라는 아이디어를 적용하려면 상당히 주의를 기울여야 한다. 객체에는 보통 상태가 있다. 실제로 객체지향 프로그래밍의 모든 목적이 상태를 클래스 저으이 안에 캡슐화하는 것이라고 주장할 수도 있다. 컬렉션을 처리하기 위해 파이썬의 클래스 선언을 사용하는 경우, 함수형프로그래밍이라는 서로 반대되는 방향으로 자신이 나뉘는 것을 느낄 수도 있다.

Callable을 사용해 합성 함수를 만들면, 만들어진 합성 함수를 사용할 때 좀 더 단순한 구문을 사용할 수 있다는 장점이 있다. 반복 가능한 매핑이나 축약으로 작업하는 경우, 상태가 있는 객체를 어떻게 도입하고, 왜 도입해야 하는지를 잘 알아야만 한다.

### 디자인 패턴 살펴보기

고차 함수를 쉽게 사용할 수 있게 해주는 람다 형식을 살펴봤다. 람다를 사용하는 경우 얻을 수 있는 큰 이점은 람다가 함수형 패러다임에 매우 가깝다는 것이다. 전통적인 함수를 작성하는 경우에는 명령형 프로그램을 작성할 수도 있고 그로 인해 더 간결하며 표현력이 뛰어난 함수형 설계에도 지장이 있을 수 있다.